<a href="https://colab.research.google.com/github/ramrajv/Gen/blob/main/Trajectory_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 8.0 MB/s 
     |████████████████████████████████| 6.3 MB 31.2 MB/s 
     |████████████████████████████████| 16.7 MB 511 kB/s 


In [4]:
!pip install movingpandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 3.1 MB 13.0 MB/s 
     |████████████████████████████████| 10.8 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 10.8 MB 17.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 10.8 MB 15.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 10.8 MB 34.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 12.1 MB 23.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |█████

In [6]:
!pip install keplergl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.7 MB 7.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for keplergl: filename=keplergl-0.3.2-py2.py3-none-any.whl size=17922344 sha256=de955c5028f7b9aaad356e9a382ffa933fee71189dec1d2ac1a30652ee65bbba
  Stored in directory: /root/.cache/pip/wheels/4b/19/e5/0852b7888ee350e9c57087ccc61eeab9daf50f5caac9c690ec
Successfully built keplergl


## 1. Layer creation with Geopandas

We are going to do the whole analysis in one Jupyter Notebook so it is efficient to import all the python libraries we are going to use.

In [7]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
from shapely.geometry import Point
from pyproj import CRS
from keplergl import KeplerGl
from datetime import datetime, timedelta

In [8]:
# Reading file
fp = r'/content/drive/MyDrive/Data/AIS_2022_03_31.csv'
data = pd.read_csv(fp)
# Creating a geometry column as Point geometry
data['geometry'] = [Point(long, lat) for long, lat in zip(data['LON'].to_list(), data['LAT'].to_list())]
# Creating a Geodataframe. Be aware it is CRS 4326 WGS84
geodata = gpd.GeoDataFrame(data, crs = CRS.from_epsg('4326'))

/usr/local/lib/python3.7/dist-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


## 2. Stop detection with Movingpandas

The first main step, before creating the trajectories in Movingpandas, is to add the timestamp as an index in the geodata

In [9]:
# Create timestamp with Pandas datetime. Inclue None as timezone
geodata['t'] = pd.to_datetime(geodata['BaseDateTime']).dt.tz_localize(None)
# Set timestamp as Index
geodata = geodata.set_index('t')
geodata.head()

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TransceiverClass,geometry
t,,,,,,,,,,,,,,,,,,
2022-03-31 00:00:01,367702220,2022-03-31T00:00:01,29.78763,-95.08070,0.1,226.5,340.0,JOE B WARD,NaN,WDI4808,31.0,12.0,21.0,8.0,NaN,57.0,A,POINT (-95.08070 29.78763)
2022-03-31 00:00:01,671226100,2022-03-31T00:00:01,25.77626,-80.20320,3.2,143.7,511.0,RELIANCE II,IMO9221322,5VHS7,79.0,0.0,52.0,12.0,2.5,70.0,A,POINT (-80.20320 25.77626)
2022-03-31 00:00:01,367767250,2022-03-31T00:00:01,29.31623,-94.78829,4.5,228.1,511.0,GLEN K,NaN,WDJ3358,52.0,0.0,0.0,0.0,0.0,52.0,A,POINT (-94.78829 29.31623)
2022-03-31 00:00:03,338327436,2022-03-31T00:00:03,47.29634,-122.42233,0.0,360.0,511.0,COOL KAT,IMO0000000,NaN,36.0,NaN,15.0,3.0,NaN,NaN,B,POINT (-122.42233 47.29634)
2022-03-31 00:00:06,367452810,2022-03-31T00:00:06,29.32824,-94.77391,2.6,319.2,511.0,JOHN W JOHNSON,IMO9602344,WDF4516,60.0,0.0,80.0,19.0,3.0,60.0,A,POINT (-94.77391 29.32824)


At this point, you can analyze some characteristics of the dataset like the length and the total vessels. The time range of the tracking is a single day.

With Movingpandas, you can create a TrajectoryCollection specifying the ID of the trajectories in this case the column with the name of the vessel name.

In [12]:
geodata.head()

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TransceiverClass,geometry
t,,,,,,,,,,,,,,,,,,
2022-03-31 00:00:01,367702220,2022-03-31T00:00:01,29.78763,-95.08070,0.1,226.5,340.0,JOE B WARD,NaN,WDI4808,31.0,12.0,21.0,8.0,NaN,57.0,A,POINT (-95.08070 29.78763)
2022-03-31 00:00:01,671226100,2022-03-31T00:00:01,25.77626,-80.20320,3.2,143.7,511.0,RELIANCE II,IMO9221322,5VHS7,79.0,0.0,52.0,12.0,2.5,70.0,A,POINT (-80.20320 25.77626)
2022-03-31 00:00:01,367767250,2022-03-31T00:00:01,29.31623,-94.78829,4.5,228.1,511.0,GLEN K,NaN,WDJ3358,52.0,0.0,0.0,0.0,0.0,52.0,A,POINT (-94.78829 29.31623)
2022-03-31 00:00:03,338327436,2022-03-31T00:00:03,47.29634,-122.42233,0.0,360.0,511.0,COOL KAT,IMO0000000,NaN,36.0,NaN,15.0,3.0,NaN,NaN,B,POINT (-122.42233 47.29634)
2022-03-31 00:00:06,367452810,2022-03-31T00:00:06,29.32824,-94.77391,2.6,319.2,511.0,JOHN W JOHNSON,IMO9602344,WDF4516,60.0,0.0,80.0,19.0,3.0,60.0,A,POINT (-94.77391 29.32824)


In [15]:
gpd = geodata.groupby('MMSI')['MMSI'].count()

In [20]:
gpd = gpd.sort_values(ascending = False)

In [30]:
gpd = gpd[gpd>1000]

In [31]:
gpd

MMSI
367458840    1280
368179250    1273
338358775    1269
338199855    1264
369970147    1255
             ... 
368121540    1003
368048990    1003
367672370    1003
316009844    1001
367799580    1001
Name: MMSI, Length: 2612, dtype: int64

In [34]:
mmsi = gpd.index

In [40]:
geodata = geodata[geodata.groupby('MMSI')['MMSI'].transform('size')>1000]

In [41]:
# Create a Trajectory Collection with Movingpandas
traj_collection = mpd.TrajectoryCollection(geodata, 'MMSI')

identify the stops of the vessels in a modest search radius at a local level of 1000 meters, with a stop duration of 4 hours or more.

In [42]:
geodata.to_csv('/content/drive/MyDrive/Data/AIS_2022_03_31_sm.csv')

In [ ]:
# Define parameters in Hours and Search radius in meters
Hours = 4
SearchRadio = 1000
# stop detection
stops = mpd.TrajectoryStopDetector(traj_collection).get_stop_segments(min_duration=timedelta(hours=Hours), max_diameter=SearchRadio)

To get a point map as a final output, we are going to use the start point of the stop. At this point is where we aggregate the stop duration which basically means the total time spent in the stop (trajectory segment). To aggregate the stop duration in the point we create a new geodataframe where we include the attribute. The code is next:

In [ ]:
# Create a new Geodataframe and define geometry column
stops_start = gpd.GeoDataFrame(columns = ['geometry'])
stops_start = stops_start.set_geometry('geometry')
# Add the ID of each stop track and define it as index
stops_start['stop_id'] = [track.id for track in stops.trajectories]
stops_start= stops_start.set_index('stop_id')
# Iteration over the Stop Trajectories
for stoptrack in stops.trajectories:
    
    # add stop duration in hours
    stops_start.at[stoptrack.id,'duration_h'] =stoptrack.get_duration().total_seconds()/3600
    
    # add length
    stops_start.at[stoptrack.id, 'length_m']=stoptrack.get_length()
    
    # add bird name
    stops_start.at[stoptrack.id, 'bird']=stoptrack.id.split(’_’)[0]
    
    # add datetime
    stops_start.at[stoptrack.id, 'datetime']= pd.to_datetime(stoptrack.id.split(’_’)[1]).tz_localize(None)
    
    # geometry with start point
    stops_start.at[stoptrack.id, 'geometry'] = stoptrack.get_start_location()
    
stops_start.head()